# 漢字・アルファベットからカタカナを推測するモデルを用いてファインチューニングする
学習済みの，漢字カナモデルを用いて，追加データを使ってファインチューニングする。

配布している[学習済みモデル](https://kktg.digital.go.jp/support/resources/index.html)は，学習の際に，
入力の文字（漢字・アルファベットなど）は一文字ずつに分割し，教師データとなる，出力の文字（カタカナ）も一文字ずつに分割して，学習する。



## 学習済みモデルの取得
[学習済みモデル](https://kktg.digital.go.jp/support/resources/index.html)の配布サイトからモデルをダウンロードする。

In [1]:
!mkdir -p model_ft/
%cd model_ft
!rm -rf *.pt
!curl -O https://kktg.digital.go.jp/public/core/1.6.1o/ai/checkpoint_best.pt 

/Users/utsubo-katsuhiko/src/kanjikana-model/train/model_ft


/Users/utsubo-katsuhiko/.pyenv/versions/3.12.8/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  991M  100  991M    0     0  19.8M      0  0:00:49  0:00:49 --:--:-- 19.1M00:30 22.8M


In [2]:
%cd ..

/Users/utsubo-katsuhiko/src/kanjikana-model/train


## 現在のモデルの出力を確認

kkt.generate('田中五郎','タカカゴロウ')

[Result(search_type=greedy,src_sentence=田中五郎,tgt_sentence=タカカゴロウ,pred_sentence=タナカゴロウ,pred_prob=-0.00013673483044840395)]



- search_type    
  greedyサーチ
- src_sentence    
  入力した漢字姓名
- tgt_sentence   
  漢字姓名に対するカタカナ姓名
- pred_sentence    
  モデルを用いてgreedyサーチで，漢字姓名から推計したカタカナ姓名
- pred_prob   
  greedyサーチで推計したカタカナ姓名の確率

In [3]:
from generate import KanjiKanaTransformerTest, Result, Args

In [4]:
args=Args()
args.model_file="model_ft/checkpoint_best.pt"
kkt=KanjiKanaTransformerTest(args)


In [5]:
kkt.generate('田中五郎','タカカゴロウ')

[Result(search_type=greedy,src_sentence=田中五郎,tgt_sentence=タカカゴロウ,pred_sentence=タナカゴロウ,pred_prob=-0.00013673483044840395)]

In [6]:
kkt.generate('月見里','ヤマナシ')

[Result(search_type=greedy,src_sentence=月見里,tgt_sentence=ヤマナシ,pred_sentence=ツキミサト,pred_prob=-0.224360853433609)]

In [7]:
kkt.generate('春夏冬','アキナシ')

[Result(search_type=greedy,src_sentence=春夏冬,tgt_sentence=アキナシ,pred_sentence=ハルナツフユ,pred_prob=-0.15543559193611145)]

In [8]:
kkt.generate('勅使河原','テシガワラ')

[Result(search_type=greedy,src_sentence=勅使河原,tgt_sentence=テシガワラ,pred_sentence=テシガワラ,pred_prob=-0.09489940106868744)]

In [9]:
kkt.generate('小鳥遊','タカナシ')

[Result(search_type=greedy,src_sentence=小鳥遊,tgt_sentence=タカナシ,pred_sentence=タカナシ,pred_prob=-0.0002544068265706301)]

## データセットを用意する


In [10]:
dataseed=[['月見里','ヤマナシ'],['春夏冬','アキナシ'],['水卜','ミウラ'],['小鳥遊','タカナシ'],['勅使河原','テシガワラ'],['大豆生田','オオマメウダ'],['東海林','ショウジ']]

In [11]:
import random
dataset=[]
for i in range(30000):
    idx=random.randint(0,len(dataseed)-1)
    dataset.append(dataseed[idx])

In [12]:
!mkdir -p dataset_ft

In [13]:

with open('dataset_ft/train.src','w',encoding='utf-8') as f:
    for d in dataset:
        f.write(d[0]+"\n")
        

In [14]:

with open('dataset_ft/train.tgt','w',encoding='utf-8') as f:
    for d in dataset:
        f.write(d[1]+"\n")
        

In [15]:
# jsonl形式に変換する

!python format.py --src dataset_ft/train.src --tgt dataset_ft/train.tgt --outfile dataset_ft/train.jsonl --src_key kanji --tgt_key kana

## validation data
検証用データをダウンロードする

In [16]:
%cd dataset_ft

/Users/utsubo-katsuhiko/src/kanjikana-model/train/dataset_ft


In [17]:
!curl -O https://kktg.digital.go.jp/public/core/1.6.1o/dataset/valid.jsonl 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1922k  100 1922k    0     0  12.1M      0 --:--:-- --:--:-- --:--:-- 12.1M


In [18]:
%cd ..

/Users/utsubo-katsuhiko/src/kanjikana-model/train


## ファインチューニングをする

In [19]:
!mkdir -p model_ft
!mkdir -p logs_ft


In [20]:
# 訓練実行

!python ./char_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 128 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 100 \
  --device mps \
  --earlystop_patient 3 \
  --output_dir model_ft \
  --tensorboard_logdir logs_ft \
  --prefix translation \
  --source_lang kanji \
  --target_lang kana \
  --train_file dataset_ft/train.jsonl \
  --valid_file dataset_ft/valid.jsonl  


load:model_ft/checkpoint_best.pt,best_epoch=89,best_loss=None
num_epochs:100
Epoch: 90, Train loss: 0.016, Val loss: 0.441, Epoch time = 81.506s
Epoch: 91, Train loss: 0.000, Val loss: 0.501, Epoch time = 83.903s
Epoch: 92, Train loss: 0.000, Val loss: 0.580, Epoch time = 102.219s
Epoch: 93, Train loss: 0.000, Val loss: 0.685, Epoch time = 95.069s
Epoch: 94, Train loss: 0.000, Val loss: 0.709, Epoch time = 82.705s


In [21]:
args=Args()
args.model_file='model_ft/checkpoint_best.pt'
kkt_ft=KanjiKanaTransformerTest(args)

In [22]:
kkt_ft.generate('田中五郎','タカカゴロウ')

[Result(search_type=greedy,src_sentence=田中五郎,tgt_sentence=タカカゴロウ,pred_sentence=タナカゴロウ,pred_prob=-7.379136513918638e-05)]

In [23]:
kkt_ft.generate('月見里','ヤマナシ')

[Result(search_type=greedy,src_sentence=月見里,tgt_sentence=ヤマナシ,pred_sentence=ヤマナシ,pred_prob=-2.181537274736911e-05)]

In [24]:
kkt_ft.generate('春夏冬','アキナシ')

[Result(search_type=greedy,src_sentence=春夏冬,tgt_sentence=アキナシ,pred_sentence=アキナシ,pred_prob=-2.7656669772113673e-05)]

## 結果
事前学習済みモデルを用いて，難読名を学習データとしてファインチューニングすることで，モデルに難読名が追加された